In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import glob
from reprojection import *

In [2]:
files = sorted(glob.glob('/home/ulyanov/data/solo/phi/polar/*'))

Reading and showing the data

In [3]:
file = files[100]

with fits.open(file) as hdul:
    header = hdul[0].header
    data = hdul[0].data


In [4]:
plt.figure(figsize=(10,10))
plt.imshow(data, origin='lower', cmap='seismic', vmin=-40, vmax=40)
plt.tight_layout()

Mapping and showing the data in spherical coordinates

In [8]:
data1 = remap(data, header, dlat=0.2, dlon=0.2, correct_mu=True, correct_dr=False, mu_thr=0.1)

In [9]:
def show_data(data, figsize=(15,8), label=r'$B_{r}$, G', cmap='seismic', vmin=-50, vmax=50, **kwargs):
    nx, ny = data.shape
    dx, dy = nx // 12, ny // 12

    fig, ax = plt.subplots(figsize=figsize)
    im = ax.imshow(data, origin='lower', cmap=cmap, vmin=vmin, vmax=vmax, **kwargs)

    ax.set_xticks(np.arange(0, ny + 1, dy), np.arange(-180,181,30))
    ax.set_yticks(np.arange(0, nx + 1, dx), np.arange(-90,91,15))

    ax.set_xlabel('Longitude, degrees')
    ax.set_ylabel('Latitude, degrees')

    cax = ax.inset_axes((0.03, 0.1, 0.1, 0.015))
    fig.colorbar(im, cax=cax, orientation='horizontal', label=label)

    ax.grid(True, ls='--', color='black', alpha=0.2)
    plt.tight_layout()

In [10]:
show_data(data1, vmin=-100, vmax=100)

nx, ny = data1.shape
nx, ny

(901, 1800)

Calculating the transformation from spherical coordinates to the original view

In [11]:
view = View.from_header(header)

transform = ~Translate((nx // 2, ny // 2)) + Scale(0.2)
transform = view.to_spherical(correct_mu=True, mu_thr=0.1) - transform

grid = np.mgrid[:view.nx, :view.ny]
grid, alpha = transform(grid)

data2 = bilinear(data1, *grid) * alpha

In [12]:
plt.figure(figsize=(10,10))
plt.imshow(data2, origin='lower', cmap='seismic', vmin=-40, vmax=40)
plt.tight_layout()

In [13]:
plt.figure(figsize=(10,10))
plt.imshow(data2 - data, origin='lower', cmap='seismic', vmin=-40, vmax=40)
plt.tight_layout()

Constructing a polar view and calculating transformation from spherical coordinates to this view

In [14]:
view = View(1024,1024,511.5,511.5,512,-90,90,0)

transform = ~Translate((nx // 2, ny // 2)) + Scale(0.2)
transform = view.to_spherical(correct_mu=True, mu_thr=0.1) - transform

grid = np.mgrid[:view.nx, :view.ny]
grid, alpha = transform(grid)

data3 = bilinear(data1, *grid) * alpha

In [15]:
def show_polar_data(data, figsize=(10,10), label='', **kwargs):
    fig = plt.figure(figsize=figsize)

    axes_coords = (0.05, 0.05, 0.9, 0.9)

    ax_image = fig.add_axes(axes_coords)
    im = ax_image.imshow(data, origin='lower', **kwargs)

    ax_image.axis('off')

    cax = ax_image.inset_axes((0.8, 0.985, 0.2, 0.015))
    fig.colorbar(im, cax=cax, orientation='horizontal', label=label)

    ax_polar = fig.add_axes(axes_coords, projection = 'polar')
    ax_polar.patch.set_alpha(0)

    lats = np.arange(15,91,15)

    ax_polar.set_rorigin(0)
    ax_polar.set_rticks(np.cos(lats * np.pi / 180))
    ax_polar.set_rlabel_position(-90)
    ax_polar.set_yticklabels([rf'{lat}$\degree$' for lat in lats], color='black')

    ax_polar.grid(True, ls='--', color='black', alpha=0.6)

    return fig, ax_image


In [16]:
show_polar_data(data3, vmin=-40, vmax=40, cmap='seismic')

(<Figure size 1000x1000 with 2 Axes>, <Axes: >)